### # Para realizar a leitura de arquivos externos deve-se importar requests (para salvar no DBFS) ou Pandas (para arquivos menores) sem precisar salvar no DBFS do Databricks


In [ ]:
import requests

caminhoParaOArquivo = "https://raw.githubusercontent.com/databricks/spark-csv/master/src/test/resources/cars.csv"
requisicao = requests.get(caminhoParaOArquivo)

# salva no DBFS
with open("/Volumes/workspace/default/arquivos-aula/cars.csv", "wb") as arquivo:
    arquivo.write(requisicao.content)

# agora lê com Spark
df_daWeb = spark.read.csv("dbfs:/Volumes/workspace/default/arquivos-aula/cars.csv", header=True, inferSchema=True)
display(df_daWeb)

### # Lendo arquivos externos com Pandas (mais fácil porque não precisa salvar no DBFS)

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/databricks/spark-csv/master/src/test/resources/cars.csv"
pdf = pd.read_csv(url)   # pandas lê direto da URL

# converte para Spark
df = spark.createDataFrame(pdf)
df.display()

### # Renomeando as colunas do arquivo da Web e excluíndo coluna inteiramente nula (Null)

In [ ]:
df_daWeb = df_daWeb.withColumnRenamed("year", "Ano")\
                   .withColumnRenamed("make", "Fabricante")\
                   .withColumnRenamed("model", "Modelo")\
                   .withColumnRenamed("comment", "Comentário")\
                   .drop("blank") # Exclui a coluna "blank"

display(df_daWeb)

### # Preenchendo celula com valor Null em uma coluna

In [ ]:
df_daWeb = df_daWeb.fillna({"Comentário": "Sem comentário"})

display(df_daWeb)

### # Substituir valor de uma célula NÃO 'NULL' em uma coluna

In [ ]:
from pyspark.sql.functions import when, col # Importa o when do SQL functions

df_daWeb = df_daWeb.withColumn(
    "Modelo", # Nome da coluna
    when(col("Modelo") == "S", "Séries S").otherwise(col("Modelo")) # Quando o modelo for "S", substitui por "Séries S", se não for 'S' mantém o modelo que já estava
)

display(df_daWeb)

### # Para substituir qualquer valor existente na coluna inteira (independente do que seja), basta usar lit (literal):

In [ ]:
from pyspark.sql.functions import lit

df_daWeb = df_daWeb.withColumn("Comentário", lit("Sem comentários dos usuários."))

display(df_daWeb)